In [ ]:
import torch
import torch_xla.runtime

x_g = torch.randn(3, 3, requires_grad=True)
y_g = torch.randn(3, 3, requires_grad=True)
z_g= torch.randn(3, 3, requires_grad=True)

x_ng = torch.randn(3, 3, requires_grad=False)
y_ng = torch.randn(3, 3, requires_grad=False)
z_ng= torch.randn(3, 3, requires_grad=False)

x = torch.randn(3, 3)
y = torch.randn(3, 3)
z = torch.randn(3, 3)

with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('ab,bc->ab', x_g.to('xla').requires_grad_(), y_g.to('xla').requires_grad_())

Einsum: Call native function
EinSum equation is not correct
Einsum: Call tensor method call
Einsum: Build lowered eisum


In [3]:
import torch_xla.debug.metrics as met

# For short report that only contains a few key metrics.
# print(met.short_metrics_report())
# For full report that includes all metrics.
print(met.metrics_report())

Metric: IrValueTensorToXlaData
  TotalSamples: 4
  Accumulator: 249.340us
  ValueRate: 008.325us / second
  Rate: 0.133555 / second
  Percentiles: 1%=015.000us; 5%=015.000us; 10%=015.000us; 20%=015.000us; 50%=053.120us; 80%=164.990us; 90%=164.990us; 95%=164.990us; 99%=164.990us
Metric: LazyTracing
  TotalSamples: 14
  Accumulator: 003ms535.180us
  ValueRate: 084.646us / second
  Rate: 0.467437 / second
  Percentiles: 1%=011.440us; 5%=011.440us; 10%=012.420us; 20%=018.630us; 50%=063.980us; 80%=215.560us; 90%=683.360us; 95%=918.630us; 99%=918.630us
Metric: TensorToData
  TotalSamples: 4
  Accumulator: 196.870us
  ValueRate: 006.573us / second
  Rate: 0.133555 / second
  Percentiles: 1%=013.770us; 5%=013.770us; 10%=013.770us; 20%=013.770us; 50%=050.210us; 80%=117.950us; 90%=117.950us; 95%=117.950us; 99%=117.950us
Counter: CreateXlaTensor
  Value: 6
Counter: DestroyLtcTensor
  Value: 3
Counter: DestroyXlaTensor
  Value: 3
Counter: RegisterXLAFunctions
  Value: 1
Counter: xla::_copy_from
  

## Einsum

In [5]:
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))
# print(torch_xla._XLAC._get_xla_tensors_hlo([out_einsum]))

IR {
  %0 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %1 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %2 = f32[3,3]{0,1} aten::einsum(%1, %0), xla_shape=f32[3,3]{0,1}, ROOT=0
}



In [4]:
with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('ab,bc->ac', x_g.to('xla'), y_g.to('xla'))
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

Einsum: Call native function
Einsum: Call tensor method call
Einsum: Build lowered eisum
IR {
  %0 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %1 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %2 = f32[3,3]{1,0} aten::einsum(%1, %0), xla_shape=f32[3,3]{1,0}, ROOT=0
}



In [5]:
with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('ab,bc->ac', x_ng.to('xla'), y_ng.to('xla'))
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

Einsum: Call native function
IR {
  %0 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %1 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %2 = f32[3,3]{1,0} aten::einsum(%1, %0), xla_shape=f32[3,3]{1,0}, ROOT=0
}

Einsum: Call tensor method call
Einsum: Build lowered eisum


In [8]:
with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('ab,bc->ac', x.to('xla'), y.to('xla'))

print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

Einsum: Call native function
IR {
  %0 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %1 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %2 = f32[3,3]{1,0} aten::einsum(%1, %0), xla_shape=f32[3,3]{1,0}, ROOT=0
}

Einsum: Call tensor method call
Einsum: Build lowered eisum


In [6]:
with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('ab,bc->ac', x_ng.to('xla'), y_ng.to('xla'))
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

Einsum: Call native function
IR {
  %0 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %1 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %2 = f32[3,3]{1,0} aten::einsum(%1, %0), xla_shape=f32[3,3]{1,0}, ROOT=0
}

Einsum: Call tensor method call
Einsum: Build lowered eisum


In [7]:
with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('...a,ab->...b', x.to('xla'), y.to('xla'))
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

Einsum: Call native function
IR {
  %0 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %1 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %2 = f32[3,3]{1,0} aten::einsum(%1, %0), xla_shape=f32[3,3]{1,0}, ROOT=0
}

Einsum: Call tensor method call
Einsum: Build lowered eisum


## New set of tests to find actual error behavior

In [14]:
import torch
import torch_xla
import torch_xla.runtime
device = torch_xla.device()

print("-- Test1")
x_1= torch.randn(3, 3, requires_grad=False).to('xla')
y_1= torch.randn(3, 3, requires_grad=False).to('xla')
with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('...m,mn->...n', x_1, y_1)
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Test2")
x_2= torch.randn(3, 3, requires_grad=True)
y_2= torch.randn(3, 3, requires_grad=True)
with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('...m,mn->...n', x_2.to('xla'), y_2.to('xla'))
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Test2.5")
x_25= torch.randn(3, 3, requires_grad=True).to('xla')
y_25= torch.randn(3, 3, requires_grad=True).to('xla')
with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('...m,mn->...n', x_25, y_25)
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Test3")
with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('...m,mn->...n', torch.randn(3, 3, requires_grad=False).to('xla'), torch.randn(3, 3, requires_grad=False).to('xla'))
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Test4")
with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('...m,mn->...n', torch.randn(3, 3, requires_grad=True).to('xla'), torch.randn(3, 3, requires_grad=True).to('xla'))
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Test5")
with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('...m,mn->...n', torch.randn(3, 3, requires_grad=True, device=device).to('xla'), torch.randn(3, 3, requires_grad=True, device=device).to('xla'))
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Test6")
x_6 = torch.randn(3, 3, requires_grad=False, device=device)
y_6 = torch.randn(3, 3, requires_grad=False, device=device)
out_einsum = torch.einsum('...n,mn->...m', x_6, y_6)
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

-- Test1
Einsum: Call native function
Einsum: Call tensor method call
Einsum: Build lowered eisum
IR {
  %0 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %1 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %2 = f32[3,3]{1,0} aten::einsum(%1, %0), xla_shape=f32[3,3]{1,0}, ROOT=0
}

-- Test2
Einsum: Call native function
Einsum: Call tensor method call
Einsum: Build lowered eisum
IR {
  %0 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %1 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %2 = f32[3,3]{1,0} aten::einsum(%1, %0), xla_shape=f32[3,3]{1,0}, ROOT=0
}

-- Test3
Einsum: Call native function
Einsum: Call tensor method call
Einsum: Build lowered eisum
IR {
  %0 = s64[] xla::device_data(), xla_shape=s64[]
  %1 = s64[] prim::Constant(), xla_shape=s64[]
  %2 = s64[] aten::mul(%1, %0), xla_shape=s64[]
  %3 = s64[] prim::Constant(), xla_shape=s64[]
  %4 = s64[] aten::add(%3, %2), xla_shape=s64[]
  %5 = s64[] prim::Constant(), xla_s

In [26]:
with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('...b,ab->...a', x_g.to('xla'), y_g.to('xla'))
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

Einsum: Call native function
Einsum: Call tensor method call
IR {
  %0 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %1 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %2 = f32[3,3]{1,0} aten::einsum(%1, %0), xla_shape=f32[3,3]{1,0}, ROOT=0
}

Einsum: Build lowered eisum


: 

In [ ]:

with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('ij,jk,kl->il', x_g.to('xla').requires_grad_(), y_g.to('xla').requires_grad_(), z_g.to('xla').requires_grad_())
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

Einsum: Call native function
EinSum cannot support operands != 2
EinSum equation is not correct
Einsum: Call tensor method call
Einsum: Build lowered eisum


F0000 00:00:1739414742.271135 1811627 debug_macros.h:21] Non-OK-status: status.status()
Status: INVALID_ARGUMENT: Expected one "," in einsum_config.
*** Begin stack trace ***
	tsl::CurrentStackTrace[abi:cxx11]()
	xla::Shape const* ConsumeValue<xla::Shape const*>(absl::lts_20230802::StatusOr<xla::Shape const*>&&)
	torch_xla::ShapeHelper::ShapeOfXlaOp(xla::XlaOp)
	torch_xla::InferOutputShape(absl::lts_20230802::Span<xla::Shape const>, std::function<xla::XlaOp (absl::lts_20230802::Span<xla::XlaOp const>)> const&)
	torch_xla::Einsum::Einsum(c10::ArrayRef<torch::lazy::Value> const&, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)
	std::shared_ptr<torch::lazy::Node> torch_xla::MakeNode<torch_xla::Einsum, std::vector<torch::lazy::Value, std::allocator<torch::lazy::Value> >&, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&>(std::vector<torch::lazy::Value, std::allocator<torch::lazy::Value> >&, std::__cxx11::basic_string<ch